# 長文書コンテンツの抽出

GPT-3は、コンテキストウィンドウに収まらない大きすぎる文書から、重要な数値、日付、その他の重要なコンテンツを抽出するのに役立ちます。これを解決するアプローチの一つは、文書をチャンクに分割し、各チャンクを個別に処理してから、一つの回答リストに結合することです。

このノートブックでは、このアプローチを実行していきます：
- 長いPDFを読み込み、テキストを抽出する
- 重要な情報を抽出するために使用するプロンプトを作成する
- 文書をチャンクに分割し、各チャンクを処理して回答を抽出する
- 最後にそれらを結合する
- このシンプルなアプローチを、より困難な3つの質問に拡張する

## アプローチ

- **セットアップ**: PDFファイル（パワーユニットに関するフォーミュラ1財務規則文書）を取得し、エンティティ抽出のためにテキストを抽出します。これを使用して、コンテンツに埋もれている回答を抽出することを試みます。
- **シンプルなエンティティ抽出**: 以下の方法で文書のチャンクから重要な情報を抽出します：
    - 質問と期待する形式の例を含むテンプレートプロンプトを作成する
    - テキストのチャンクを入力として受け取り、プロンプトと組み合わせて応答を取得する関数を作成する
    - テキストをチャンクに分割し、回答を抽出して解析用に出力するスクリプトを実行する
- **複雑なエンティティ抽出**: より困難な推論を必要とする、より難しい質問をする

## セットアップ

In [ ]:
!pip install textract
!pip install tiktoken

In [11]:
import textract
import os
import openai
import tiktoken

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

# Extract the raw text from each PDF using textract
text = textract.process('data/fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf', method='pdfminer').decode('utf-8')
clean_text = text.replace("  ", " ").replace("\n", "; ").replace(';',' ')

## シンプルなエンティティ抽出

In [20]:
# Example prompt - 
document = '<document>'
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR\n2. What is the value of External Manufacturing Costs in USD\n3. What is the Capital Expenditure Limit in USD\n\nDocument: \"\"\"<document>\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR
2. What is the value of External Manufacturing Costs in USD
3. What is the Capital Expenditure Limit in USD

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [17]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

def extract_chunk(document,template_prompt):
    prompt = template_prompt.replace('<document>',document)

    messages = [
            {"role": "system", "content": "You help extract information from documents."},
            {"role": "user", "content": prompt}
            ]

    response = client.chat.completions.create(
            model='gpt-4', 
            messages=messages,
            temperature=0,
            max_tokens=1500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
    return "1." + response.choices[0].message.content

In [ ]:
# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

results = []
    
chunks = create_chunks(clean_text,1000,tokenizer)
text_chunks = [tokenizer.decode(chunk) for chunk in chunks]

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    #print(chunk)
    print(results[-1])


In [31]:
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: USD 95,000,000 (Page 2); GBP 76,459,000 (Page 2); EUR 90,210,000 (Page 2)',
 '2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 and 31 December 2025, adjusted for Indexation (Page 10)',
 '3. What is the Capital Expenditure Limit in USD: US Dollars 30,000,000 (Page 32)']

## 複雑なエンティティ抽出

In [32]:
# Example prompt - 
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. How is a Minor Overspend Breach calculated\n2. How is a Major Overspend Breach calculated\n3. Which years do these financial regulations apply to\n\nDocument: \"\"\"<document>\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. How is a Minor Overspend Breach calculated
2. How is a Major Overspend Breach calculated
3. Which years do these financial regulations apply to

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [34]:
results = []

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. How is a Minor Overspend Breach calculated: A Minor Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by less than 5% (Page 24)',
 '2. How is a Major Overspend Breach calculated: A Material Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by 5% or more (Page 25)',
 '3. Which years do these financial regulations apply to: 2026 onwards (Page 1)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 2)',
 '3. Which years do these financial regulations apply to: 2022-2025 (Page 6)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 10)',
 '3. Which years do these financial regulations apply to: 202

## 統合

最初の2つの回答は安全に抽出することができましたが、3つ目は各ページに表示される日付によって混乱しました。ただし、正しい回答もそこに含まれています。

これをさらに調整するために、以下の実験を検討できます：
- より説明的または具体的なプロンプト
- 十分な訓練データがある場合、一連の出力を非常によく見つけるためのモデルのファインチューニング
- データをチャンクする方法 - 私たちは重複なしで1000トークンを使用しましたが、情報をセクションに分割したり、トークンで切り分けるなど、より知的なチャンキングによってより良い結果が得られる可能性があります

しかし、最小限の調整で、長い文書の内容を使用してさまざまな難易度の6つの質問に回答し、エンティティ抽出を必要とする任意の長い文書に適用できる再利用可能なアプローチを手に入れました。あなたがこれで何ができるかを楽しみにしています！